In [ ]:
import musdb
import museval
import numpy as np
import torch
from src.model import Unet
from tqdm import tqdm
import os

def evaluate_model(model, test_tracks, output_dir='./eval'):
    """
    Evaluate model on test tracks and compute SDR, SIR, SAR
    
    Args:
        model: trained Unet model
        test_tracks: list of musdb tracks
        output_dir: where to save results
    """
    os.makedirs(output_dir, exist_ok=True)
    
    all_results = []
    
    for track in tqdm(test_tracks, desc="Evaluating"):
        # 1. Separate vocals using your model
        vocal_estimate = separate_vocals(model, track)
        
        # 2. Calculate accompaniment
        accompaniment_estimate = track.audio - vocal_estimate
        
        # 3. Prepare estimates dict
        estimates = {
            'vocals': vocal_estimate,
            'accompaniment': accompaniment_estimate
        }
        
        # 4. Evaluate
        scores = museval.eval_mus_track(
            track, 
            estimates, 
            output_dir=output_dir
        )
        
        all_results.append(scores)
        
        print(f"\n{track.name}:")
        print(f"  SDR Vocal: {scores.targets['vocals'].metrics['SDR'].mean():.2f} dB")
        print(f"  SIR Vocal: {scores.targets['vocals'].metrics['SIR'].mean():.2f} dB")
        print(f"  SAR Vocal: {scores.targets['vocals'].metrics['SAR'].mean():.2f} dB")
    
    return all_results

In [ ]:
# Download iKala dataset
# https://zenodo.org/record/1287728

import ikala  # ou charge manuellement

# Load iKala
ikala_tracks = load_ikala_tracks()

# Evaluate
ikala_results = evaluate_model(model, ikala_tracks, output_dir='./eval_ikala')

# Compare
print("\nRESULTS ON IKALA:")
aggregate_results(ikala_results)